In [5]:
import pandas as pd

state_data = pd.read_csv('datasets/merged_state_data.csv')
weather_data = pd.read_csv('datasets/weather_monthly_state_aggregates.csv')
wildfire_data = pd.read_csv('datasets/wildfire_sizes_before_2010.csv')

print(len(state_data))
print(len(weather_data))
print(len(wildfire_data))

50
11775
9632


In [17]:
state_data = state_data.rename(columns={'State': 'STATE', 'mean_elevation': 'MEAN_ELEVATION', 'Land Area (sq mi)': 'LAND_AREA', 'Water Area (sq mi)': 'WATER_AREA', 'Total Area (sq mi)': 'TOTAL_AREA', 'Percentage of Federal Land': 'PERCENTAGE_FEDERAL_LAND', 'Urbanization Rate (%)': 'URBANIZATION_RATE'})
state_data.head()

,STATE,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE
0,AK,580,570641,94743,665384,60.9%,64.9
1,AL,150,50645,1775,52420,2.7%,57.7
2,AR,200,52035,1143,53179,9.4%,55.5
3,AZ,1250,113594,396,113990,38.6%,89.3
4,CA,880,155779,7916,163695,45.4%,94.2


In [89]:
weather_data = weather_data.rename(columns={'State': 'STATE', 'year_month': 'PERIOD'})
weather_data.head()

,STATE,PERIOD,PRCP,EVAP,TMIN,TMAX
0,AK,1992-05,5.36,44.17,-56.0,233.0
1,AK,1992-06,14.03,39.27,-17.0,322.0
2,AK,1992-07,16.78,30.91,28.0,300.0
3,AK,1992-08,10.46,20.14,-33.0,267.0
4,AK,1992-09,17.25,15.27,-61.0,183.0


In [90]:
weather_data.describe()

,PRCP,EVAP,TMIN,TMAX
count,11775.000000,11775.000000,11775.000000,11775.000000
mean,22.792403,62.285918,11.948875,318.659788
std,18.153277,185.920998,82.541805,71.463314
min,0.000000,-96.870000,-456.000000,-167.000000
25%,9.570000,31.280000,-39.000000,283.000000
50%,19.820000,48.400000,6.000000,328.000000
75%,32.000000,64.895000,67.000000,361.000000
max,259.000000,15445.700000,233.000000,539.000000


In [91]:
wildfire_data = wildfire_data.rename(columns={'month': 'PERIOD', 'total_fire_size': 'TOTAL_FIRE_SIZE'})
wildfire_data.head()

,STATE,PERIOD,TOTAL_FIRE_SIZE
0,AK,1992-04,10.1
1,AK,1992-05,4309.5
2,AK,1992-06,86460.6
3,AK,1992-07,48578.5
4,AK,1992-08,3321.8


In [93]:
wildfire_data.describe()

,TOTAL_FIRE_SIZE
count,9.632000e+03
mean,1.064928e+04
std,8.248342e+04
min,1.000000e-02
25%,2.400000e+01
50%,2.997650e+02
75%,2.439590e+03
max,4.779145e+06


In [94]:
weather_wildfire_data = pd.merge(wildfire_data, weather_data, on=['STATE', 'PERIOD'], how='left')
print(len(weather_wildfire_data))
# weather_wildfire_data.head()
weather_wildfire_data.describe()

9632


,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX
count,9.632000e+03,6583.000000,6583.000000,6583.000000,6583.000000
mean,1.064928e+04,21.221559,50.931341,10.677199,328.046939
std,8.248342e+04,15.636771,23.654458,73.950807,61.079450
min,1.000000e-02,0.000000,-6.860000,-456.000000,-83.000000
25%,2.400000e+01,9.775000,34.370000,-39.000000,289.000000
50%,2.997650e+02,18.790000,49.320000,-6.000000,328.000000
75%,2.439590e+03,29.480000,63.150000,56.000000,367.000000
max,4.779145e+06,213.000000,392.330000,217.000000,539.000000


In [54]:
state_weather_wildfire_data = pd.merge(weather_wildfire_data, state_data, on=['STATE'], how='left')
print(len(state_weather_wildfire_data))
state_weather_wildfire_data.head()

9632


,STATE,PERIOD,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE
0,AK,1992-04,10.1,NaN,NaN,NaN,NaN,580,570641,94743,665384,60.9%,64.9
1,AK,1992-05,4309.5,5.36,44.17,-56.0,233.0,580,570641,94743,665384,60.9%,64.9
2,AK,1992-06,86460.6,14.03,39.27,-17.0,322.0,580,570641,94743,665384,60.9%,64.9
3,AK,1992-07,48578.5,16.78,30.91,28.0,300.0,580,570641,94743,665384,60.9%,64.9
4,AK,1992-08,3321.8,10.46,20.14,-33.0,267.0,580,570641,94743,665384,60.9%,64.9


In [29]:
state_weather_wildfire_data.to_csv('datasets/combined_dataset.csv', index=False)

#### More data cleaning

In [82]:
import numpy as np
from sklearn.linear_model import LinearRegression

def impute_by_group_all(group_df, target_cols):
    for target in target_cols:
        predictors = [col for col in group_df.columns if col != target and col != "STATE"]
        train = group_df[group_df[target].notnull()]
        missing = group_df[group_df[target].isnull()]

        # only perform imputation if there is enough training data and missing values exist.
        if len(train) >= 2 and not missing.empty:
            model = LinearRegression()
            model.fit(train[predictors], train[target])
            predictions = model.predict(missing[predictors])
            group_df.loc[missing.index, target] = predictions
    return group_df

In [86]:
# load dataset
dataset_loaded = pd.read_csv('datasets/combined_dataset.csv')
dataset_loaded.head()

# convert date to sin encoded month / convert strings to numeric
dataset_loaded['PERIOD'] = pd.to_datetime(dataset_loaded['PERIOD'], format='%Y-%m')
dataset_loaded['MONTH'] = dataset_loaded['PERIOD'].dt.month
dataset_loaded['YEAR'] = dataset_loaded['PERIOD'].dt.year
dataset_loaded['MONTH_SIN'] = np.sin(2 * np.pi * dataset_loaded['MONTH'] / 12)
dataset_loaded['PERCENTAGE_FEDERAL_LAND'] = pd.to_numeric(
    dataset_loaded['PERCENTAGE_FEDERAL_LAND'].astype(str).str.rstrip('%').astype(float),
    errors='coerce'
)
dataset_loaded = dataset_loaded.drop(columns=['PERIOD'])

dataset_loaded.head()
dataset_loaded.describe()

#impute missing values
target_columns = ['PRCP', 'EVAP', 'TMIN', 'TMAX']
imputed_data = dataset_loaded.groupby('STATE', group_keys=False).apply(
    lambda group: impute_by_group_all(group, target_cols=target_columns)
)

# drop redundant cols
imputed_data = imputed_data.drop(columns=[ 'MONTH', 'YEAR'])

print(len(imputed_data))
imputed_data.head()

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [44]:
imputed_data.describe()

,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE,MONTH_SIN
count,9.632000e+03,6583.000000,6583.000000,6583.000000,6583.000000,9632.000000,9632.000000,9632.000000,9632.000000,9632.000000,9632.000000,9.632000e+03
mean,1.064928e+04,21.221559,50.931341,10.677199,328.046939,546.721034,74150.715947,5075.553156,79226.212313,16.331676,72.499201,-2.842887e-03
std,8.248342e+04,15.636771,23.654458,73.950807,61.079450,538.988922,78942.638002,12927.298639,88882.864369,20.535084,13.565915,7.162742e-01
min,1.000000e-02,0.000000,-6.860000,-456.000000,-83.000000,20.000000,1034.000000,192.000000,1545.000000,0.300000,35.100000,-1.000000e+00
25%,2.400000e+01,9.775000,34.370000,-39.000000,289.000000,180.000000,40861.000000,791.000000,42775.000000,2.400000,63.200000,-8.660254e-01
50%,2.997650e+02,18.790000,49.320000,-6.000000,328.000000,300.000000,55519.000000,1509.000000,65496.000000,5.400000,71.900000,1.224647e-16
75%,2.439590e+03,29.480000,63.150000,56.000000,367.000000,670.000000,82170.000000,4537.000000,86936.000000,28.600000,85.600000,8.660254e-01
max,4.779145e+06,213.000000,392.330000,217.000000,539.000000,2070.000000,570641.000000,94743.000000,665384.000000,80.100000,94.200000,1.000000e+00
